In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-12-22 15:32:35--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.3’

book-crossings.zip. 100%[===================>]  24.88M   146MB/s    in 0.2s    

2024-12-22 15:32:35 (146 MB/s) - ‘book-crossings.zip.3’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [5]:
# import csv data into dataframes

df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['bookId', 'title', 'author'],
    usecols=['bookId', 'title', 'author'],
    dtype={'bookId': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['userId', 'bookId', 'rating'],
    usecols=['userId', 'bookId', 'rating'],
    dtype={'userId': 'int32', 'bookId': 'str', 'rating': 'float32'})


In [6]:
# add your code here - consider creating a new cell for each section of code

# get list of users to remove
user_ratingCount = (df_ratings.
     groupby(by = ['userId'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['userId', 'totalRatingCount']]
)
users_to_remove = user_ratingCount.query('totalRatingCount > 200').userId.tolist()

# merge rating and catalog by bookId
df = pd.merge(df_ratings,df_books,on='bookId')

# create totalRatingCount
book_ratingCount = (df.
     groupby(by = ['title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['title', 'totalRatingCount']]
    )

rating_with_totalRatingCount = df.merge(book_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# remove books with less than 100 ratings
rating_popular_movie = rating_with_totalRatingCount.query('totalRatingCount > 100')

# remove from the dataset users with less than 200 ratings
rating_popular_movie = rating_popular_movie[rating_popular_movie['userId'].isin(users_to_remove)]

# drop duplicates
#rating_popular_movie.drop_duplicates(subset=['title'], keep=False, inplace=True)

# pivot table and create matrix
book_features_df = rating_popular_movie.pivot_table(index='title',columns='userId',values='rating').fillna(0)
book_features_df_matrix = csr_matrix(book_features_df.values)

In [19]:
print(book_features_df_matrix)
print()
#print(book_features_df_matrix)

  (0, 0)	9.0
  (0, 16)	8.0
  (0, 26)	3.3333332538604736
  (0, 37)	9.0
  (0, 42)	8.0
  (0, 59)	7.0
  (0, 136)	9.0
  (0, 157)	10.0
  (0, 171)	9.0
  (0, 188)	8.0
  (0, 221)	10.0
  (0, 222)	10.0
  (0, 246)	5.0
  (0, 278)	10.0
  (0, 360)	10.0
  (0, 371)	9.0
  (0, 390)	10.0
  (0, 477)	9.0
  (0, 491)	7.0
  (0, 507)	9.0
  (0, 554)	10.0
  (0, 603)	9.0
  (0, 610)	8.0
  (0, 694)	9.0
  (0, 733)	9.0
  :	:
  (903, 16)	8.0
  (903, 26)	3.5
  (903, 86)	10.0
  (903, 104)	2.5
  (903, 112)	5.0
  (903, 151)	6.0
  (903, 234)	7.0
  (903, 291)	8.0
  (903, 304)	9.0
  (903, 323)	10.0
  (903, 368)	10.0
  (903, 385)	9.0
  (903, 446)	7.0
  (903, 450)	9.0
  (903, 485)	7.0
  (903, 490)	3.0
  (903, 498)	10.0
  (903, 505)	8.0
  (903, 544)	10.0
  (903, 592)	8.0
  (903, 699)	10.0
  (903, 712)	8.0
  (903, 744)	10.0
  (903, 829)	9.0
  (903, 879)	8.0



In [20]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
    model_knn = NearestNeighbors(metric = 'cosine', n_neighbors=5, algorithm='auto')
    model_knn.fit(book_features_df_matrix)

    # found book TODO: user a better search
    for query_index in range(len(book_features_df)):
        if book_features_df.index[query_index] == book:
            break

    # creating return structure
    ret = [book_features_df.index[query_index], []]
    distances, indices = model_knn.kneighbors(book_features_df.iloc[query_index,:].values.reshape(1, -1))
    # now we located the book. lets show the recomendations
    for i in range(1, len(distances.flatten())):
        ret[1].insert(0, [book_features_df.index[indices.flatten()[i]], distances.flatten()[i]])
    return ret

In [21]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Weight of Water', 0.77085835], ['I Know This Much Is True', 0.7529293], ['The Lovely Bones: A Novel', 0.7230184], ['Blue Diary', 0.71828747]]]
You passed the challenge! 🎉🎉🎉🎉🎉
